In [1]:
import sqlite3
import pandas as pd
import numpy as np

In [2]:
connection = sqlite3.connect('sqlite_db_pythonsqlite.db')

In [3]:
cursor = connection.cursor()

/* Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! */

In [4]:
query1 = 'SELECT * FROM Bookings'
query2 = 'SELECT * FROM Facilities'
query3 = 'SELECT * FROM Members'

In [5]:
bookings = cursor.execute(query1).fetchall()
booking_columns = cursor.execute('PRAGMA table_info(Bookings);').fetchall()
facilities = cursor.execute(query2).fetchall()
facilities_columns = cursor.execute('PRAGMA table_info(Facilities);').fetchall()
members = cursor.execute(query3).fetchall()
members_columns = cursor.execute('PRAGMA table_info(Members);').fetchall()

In [6]:
booking_columns = [num[1] for num in booking_columns]
facilities_columns = [num[1] for num in facilities_columns]
members_columns = [num[1] for num in members_columns]

In [7]:
bookings_df = pd.DataFrame(bookings, index=range(len(bookings)), columns=booking_columns)
facilities_df = pd.DataFrame(facilities, index=range(len(facilities)), columns=facilities_columns)
members_df = pd.DataFrame(members, index=range(len(members)), columns=members_columns)

In [8]:
bookings_facilities_df = bookings_df.merge(facilities_df, how='outer', on='facid', suffixes=['_book', '_fac'])
bookings_facilities_df.head()

,bookid,facid,memid,starttime,slots,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,3,1,2012-07-03 11:00:00,2,Table Tennis,0.0,5.0,320,10
1,19,3,3,2012-07-05 09:00:00,2,Table Tennis,0.0,5.0,320,10
2,20,3,1,2012-07-05 19:00:00,2,Table Tennis,0.0,5.0,320,10
3,30,3,1,2012-07-06 11:00:00,2,Table Tennis,0.0,5.0,320,10
4,43,3,2,2012-07-07 12:30:00,2,Table Tennis,0.0,5.0,320,10


In [9]:
revenue = []
for i, memid in enumerate(bookings_facilities_df['memid']):
    if memid != 0:
        revenue.append(bookings_facilities_df['slots'].iloc[i] * bookings_facilities_df['membercost'].iloc[i])
    if memid == 0:
        revenue.append(bookings_facilities_df['slots'].iloc[i] * bookings_facilities_df['guestcost'].iloc[i])
        
bookings_facilities_df['revenue'] = revenue
del revenue

In [10]:
total_revenue = bookings_facilities_df[['facid', 'name', 'revenue']].groupby(['facid', 'name'])['revenue'].sum()

In [11]:
final_list = total_revenue.loc[total_revenue < 1000].sort_values(ascending=False)
print(final_list)

facid  name         
8      Pool Table       270.0
7      Snooker Table    240.0
3      Table Tennis     180.0
Name: revenue, dtype: float64


Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [12]:
members_df['recommendedby'].replace({'':np.nan}, inplace=True)
members_df['recommendedby'] = members_df['recommendedby'].astype(float)

In [21]:
recommend_df = members_df.loc[:, ['memid', 'surname', 'firstname', 'recommendedby']].merge(members_df.loc[:, ['memid', 'surname', 'firstname', 'recommendedby']],
                                                                                           left_on='memid', right_on='recommendedby', 
                                                                                           suffixes=['_origianl', '_recommended'], how='left')

In [25]:
recommend_df.sort_values(['surname_recommended', 'firstname_recommended']).head()

,memid_origianl,surname_origianl,firstname_origianl,recommendedby_origianl,memid_recommended,surname_recommended,firstname_recommended,recommendedby_recommended
17,9,Stibbons,Ponder,6.0,15.0,Bader,Florence,9.0
16,9,Stibbons,Ponder,6.0,12.0,Baker,Anne,9.0
21,13,Farrell,Jemima,NaN,16.0,Baker,Timothy,13.0
9,3,Rownam,Tim,NaN,8.0,Boothe,Tim,3.0
2,1,Smith,Darren,NaN,5.0,Butters,Gerald,1.0


Q12: Find the facilities with their usage by member, but not guests.

In [31]:
usage_df = bookings_facilities_df.loc[bookings_facilities_df['memid'] != 0, ['memid', 'name', 'slots']]
usage_df

,memid,name,slots
0,1,Table Tennis,2
1,3,Table Tennis,2
2,1,Table Tennis,2
3,1,Table Tennis,2
4,2,Table Tennis,2
...,...,...,...
4016,7,Massage Room 2,2
4024,15,Massage Room 2,2
4028,14,Massage Room 2,2
4035,22,Massage Room 2,2


In [32]:
usage_df.groupby('name')['slots'].sum()

name
Badminton Court    1086
Massage Room 1      884
Massage Room 2       54
Pool Table          856
Snooker Table       860
Squash Court        418
Table Tennis        794
Tennis Court 1      957
Tennis Court 2      882
Name: slots, dtype: int64

Q13: Find the facilities usage by month, but not guests

In [38]:
bookings_facilities_df.set_index('starttime', inplace=True)
bookings_facilities_df.set_index(pd.to_datetime(bookings_facilities_df.index), inplace=True)

In [63]:
bookings_facilities_df.loc[bookings_facilities_df['memid'] != 0, ['slots', 'name']].groupby('name').resample('M')['slots'].count()

name             starttime 
Badminton Court  2012-07-31     51
                 2012-08-31    132
                 2012-09-30    161
Massage Room 1   2012-07-31     77
                 2012-08-31    153
                 2012-09-30    191
Massage Room 2   2012-07-31      4
                 2012-08-31      9
                 2012-09-30     14
Pool Table       2012-07-31    103
                 2012-08-31    272
                 2012-09-30    408
Snooker Table    2012-07-31     68
                 2012-08-31    154
                 2012-09-30    199
Squash Court     2012-07-31     23
                 2012-08-31     85
                 2012-09-30     87
Table Tennis     2012-07-31     48
                 2012-08-31    143
                 2012-09-30    194
Tennis Court 1   2012-07-31     65
                 2012-08-31    111
                 2012-09-30    132
Tennis Court 2   2012-07-31     41
                 2012-08-31    109
                 2012-09-30    126
Name: slots, dtype: int64